<a href="https://colab.research.google.com/github/scienceofdeduction/pub/blob/master/julia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%shell
if ! command -v julia 3>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.0-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
    julia -e 'using Pkg; Pkg.add("IJulia"); Pkg.precompile();'
fi

In [26]:
%%sh
which julia

/usr/local/bin/julia


![julia colab][1]

[1]: https://user-images.githubusercontent.com/19316036/90217362-0079f380-ddb6-11ea-84d4-5059a2bea538.png

In [ ]:
using Pkg

In [24]:
%%sh
julia -e 'using Pkg; Pkg.status();'

Status `~/.julia/environments/v1.5/Project.toml`
  [587475ba] Flux v0.11.1
  [7073ff75] IJulia v1.21.2
  [bd369af6] Tables v1.0.5


In [ ]:
%%sh 
julia -e 'using Pkg; Pkg.add("Flux");'
julia -e 'using Pkg; Pkg.add("CUDA");'

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("JuliaConnectoR")

In [ ]:
%%R
require(JuliaConnectoR)

In [ ]:
%%R
# import julia into R
# modelling in R
flux <- juliaImport("Flux")
m <- Flux$Chain(
        Flux$Dense(4L, 12L, Flux$relu), 
        Flux$Dense(12L, 3L, Flux$softmax)
)

In [ ]:
%%R
# 
# wrap around julia code into function in R
# in my view, this is cleaner
#
mlp <- juliaEval('
  using CUDA, Flux
  using Flux: @epochs, onehotbatch, onehot, onecold, logitcrossentropy, mse, throttle
  function mlp(x, y)
    opt = ADAM(.01) |> gpu
    m = Chain(Dense(4, 12, relu), Dense(12, 3, softmax)) |> gpu
    loss(x, y) = crossentropy(m(x), y) 
    # @epochs 10 train!(loss, params(m), [(x, y)], opt)

  end 
')

In [25]:
%%sh
nvidia-smi

Fri Aug 14 06:04:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P8    29W / 149W |     11MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------